.. _AWS Connect:

AWS Connect
--------------


Initial steps to set up a Jupyter notebook using AWS. 

Note: Anything in <__> means you can fill in with whatever name you please. 


## Create VPC for your notebooks

We need to create a virtual private cloud (VPC) for our machine learning instances. It will span all the availability zones in the region. 
Within the VPC we will need a subnet which is located in an Availability Zone. To connect to it and run a Jupyter notebook we need to set up a route table (think your home router) and internet gateway.


1) Navigate to VPC in dashboard: 

*   You can search for VPC or find it under Networking & Content Delivery

2) This brings you to VPC Dashboard: 

![VPC Dashboard](../_static/images/vpc_dashboard.png)

3) Click `Your VPCs` and then `Create VPC`. 

4) In `Create VPC` you will need to fill in: 

![Create VPC](../_static/images/create_vpc.png) ## Redo this image. 

Name tag: \<VPC_Name\>

IPv4 CIDR block: 10.0.0.0/24

IPv6 CIDR block: None

Tenancy: Default


## Create Internet Gateway

1) Navigate in left menu to `Internet Gateways`

2) Create Internet Gateway

3) Enter:
*   Name: \<IG_Name\>

4)  Navigate to actions:
*   Attach to VPC: \<VPC_Name\>

## Create Routing Table

1) Navigate in left menu to `Route Tables`

2) Create route table

3) Enter:
*   Name: \<RT_Name\>
*   VPC: Attach \<VPC_Name\>

4) Navigate to actions and select edit routes:
*  Destination: 0.0.0.0/0
*  Target: \<IG_Name\>

## Create subnet

1) Navigate in left menu to `Subnets`

2) Create New Subnet

3) Enter:
*   Name: <VPC_sn1>
*   VPC: <VPC_Name>
*   Availability Zone: No preference
*   IPv4: 10.0.0.0/24

4) Navigate to actions and select Edit Route Table Associations:
*   Route Table ID: \<RT_Name\>

5) Navigate to actions and select Modify auto-assign IP settings:
*   Select auto-assign IPv4. Enable auto-assign public IPv4 address

## Set up Security Groups

1) Navigate in left menu to `Security` -> `Security Groups`:

![Nav to Security Group](../_static/images/create_SG.png)

2) Fill in Security group name, description and connect it to your VPC you created before: 

![Create Security Group](../_static/images/create_SG_2.png)

3) Navigate to Inbound Rules and Edit Rules:
Add these rules:
*   SSH/TCP/22/Anywhere/ 0.0.0.0/0

*   HTTPS/TCP/443/Anywhere/ 0.0.0.0/0

*   Custom TCP/TCP/8888/Anywhere/ 0.0.0.0/0

## Launch EC2 instance

1) Navigate to EC2

2) Click `Launch Instance`

3) Scroll down and select UbuntuServer 18

![select machine image](../_static/images/machine_image.png)

4) Select t2.micro 
*   Note: You will most likely not be using t2.micros. For GPUs you will need to send a request to AWS to enable P2s for you. 

5) Configure Instance Details:

![configure instance](../_static/images/conf_instance.png)

*   Number of instances: 1

*   Network: \<VPC_Name\>

*   Subnet: \<Subnet_Name\>

*   Auto-assign Public IP: Enable (I've had trouble with it not assigning public IPs here so automatically enable regardless of set up in Subnet. 

6) Add storage:

*   For this tutorial leave as is

7)  Configure Security Group:

*   Select an existing security group
*   Security group name: \<SG_Name\>

8) Review and launch


9) Create new key
*   Download Key

## Instance Spins Up

While the instance is spinning up feel free to move your new key to the .ssh directory (if using Linux)

`cd ~/.ssh`

`mv ~/Downloads/<key_name>.pem <key_name>.pem`

`chmod 400 <key_name>.pem`


## Connect to Instance

1)  Navigate to Instances and select Connect

2)  Copy the provided terminal line to connect. It should look like:

![connect to instance](../_static/images/connect_to_instance.png)

Part intentionally blacked out for security reasons. 

3)  Paste into terminal with CTRL+SHIFT+C. Make sure you are still in .ssh directory. 

4)  Success!

## Install Anaconda

This section is performed in terminal.

1) Ensure ec2 instance is up to date. You will typically see when logging in:

![connect to instance](../_static/images/connected_to_image.png)

2) In your browser navigate to: 
https://repo.continuum.io/archive

3) Find the most recent Anaconda3 distro for Linux_x86_64 and copy the address of the file

4) In your terminal connection to the ec2 instance type:
`wget <Anaconda3_whatever_Linux_x86_64.sh>`

5) Type: `bash <Anaconda3_whatever_Linux_x86_64.sh>`

6) Hit enter multiple times and then when prompted type yes. Then hit enter to confirm location

7) When prompted for prepending Anaconda3 to PATH select yes.

If not prompted type `export PATH=/home/ubuntu/anaconda3/bin:$PATH` once install is completed. 

8) Confirm python is anaconda:

`which python`

## Set up Jupyter Notebook

1) Type: `jupyter notebook --generate-config`


##### Create certificates for https
2) Type: `mkdir certs`

3) Type: `cd certs`

4) Type: `sudo openssl req -x509 -nodes -days 365 -newkey rsa:1024 -keyout mycert.pem -out mycert.pem`

5) Type: `sudo chmod 777 mycert.pem`

6) Navigate to .jupyter directory: `cd ~/.jupyter/`

7) Type: `vi jupyter_notebook_config.py`

8) Enter at the top: 
`c = get_config() 
c.NotebookApp.certfile = u'~/certs/mycert.pem' # Run on all IP addresses of your instance
c.NotebookApp.ip = '*'# Don't open browser by default
c.NotebookApp.open_browser = False  # Fix port to 8888
c.NotebookApp.port = 8888`

8) Exit vi: `esc` and `wq`

9) Create dir for notebooks: 
`cd ~`
`mkdir nbs`

10) Launch a notebook: `jupyter notebook`

11) Copy the location of your notebook: `https://<ip_address>:8888/?token=sometoken`

## Launch notebook

1) Navigate to Instances and locate your Public IP or DNS address. 

2) Copy into your browser the location from above and replace the IP address with the public IP address. 

3) Success!

## Shut down your notebook!

1) Select your instance, select actions, instance state, and select stop. 


Congrats. You now have the knowledge to host a Jupyter Notebook on an ec2 instance. This will save you significant money from using SageMaker.